In [2]:
import os
import h5py
import torch
from torch.utils.data import Dataset,DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets,transforms
from torchvision.transforms import ToTensor
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import argparse
import numpy as np
import time
from copy import deepcopy # Add Deepcopy for args
from PIL import ImageOps

import matplotlib.pyplot as plt

In [8]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, ConcatDataset

train_dir = r"C:\Users\roobe\PKG - HER2 tumor ROIs_v3\Yale_trastuzumab_response_cohort\patches\train2"
val_dir = r"C:\Users\roobe\PKG - HER2 tumor ROIs_v3\Yale_trastuzumab_response_cohort\patches\test2"

class HistogramEqualization:
    def __call__(self, img):
        return ImageOps.equalize(img)  # 이미지 대비를 자동으로 맞춤

base_transform = transforms.Compose([
    transforms.Resize((224, 224)),       # 크기 조정
    transforms.ToTensor(),               # 이미지 -> 텐서
    transforms.Normalize([0.5]*3, [0.5]*3)  # 정규화
])



# 증강 transform
aug_transform = transforms.Compose([
    HistogramEqualization(),          # Histogram Equalization 먼저 적용->명암 균형 정규화
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.1),# 밝기, 대비, 채도, 색조 무작위 조정
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# ImageFolder로 원본 및 증강 데이터셋 생성
base_dataset = datasets.ImageFolder(root=train_dir, transform=base_transform)
aug_dataset = datasets.ImageFolder(root=train_dir, transform=aug_transform)

# 두 데이터셋 합치기 → 이미지 수 2배
train_dataset = ConcatDataset([base_dataset] + [aug_dataset]*5)

# DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# 검증 데이터셋은 증강 없이
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])
val_dataset = datasets.ImageFolder(root=val_dir, transform=val_transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# 클래스 확인
print("클래스 목록:", base_dataset.classes)

클래스 목록: ['nonresponder', 'responder']


In [9]:
train =r"C:\Users\roobe\PKG - HER2 tumor ROIs_v3\Yale_trastuzumab_response_cohort\patches\train2\nonresponder"
exts = ('.png', '.jpg', '.jpeg')  # 이미지 확장자

count = len([f for f in os.listdir(train) if f.lower().endswith(exts)])
print(f"클래스 'nonresponder' 폴더의 이미지 수: {count}")

클래스 'nonresponder' 폴더의 이미지 수: 9727


In [10]:
train =r"C:\Users\roobe\PKG - HER2 tumor ROIs_v3\Yale_trastuzumab_response_cohort\patches\train2\responder"
exts = ('.png', '.jpg', '.jpeg')  # 이미지 확장자

count = len([f for f in os.listdir(train) if f.lower().endswith(exts)])
print(f"클래스 'nonresponder' 폴더의 이미지 수: {count}")

클래스 'nonresponder' 폴더의 이미지 수: 5801


In [11]:
print(f"전체 데이터 개수: {len(train_dataset)}")

전체 데이터 개수: 93168


In [13]:
# device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print('Current device is', device)

# for reproducibility
torch.manual_seed(2023)
if device == 'cuda':
    torch.cuda.manual_seed_all(2023)

Current device is cuda


In [22]:
import torch
import torch.nn as nn

class CNN(nn.Module):  #resnet18로 바꿔서도 해보기
    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=2),  # (3, 224, 224) -> (32, 113, 113)
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)                  # -> (32, 56, 56)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1), # -> (64, 28, 28)
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)                  # -> (64, 14, 14)
        )

        self.dropout = nn.Dropout2d(p=0.5)

        self.fc = nn.Linear(64 * 14 * 14, 2)
        nn.init.kaiming_normal_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.dropout(out)
        out = out.view(out.size(0), -1)   # flatten
        out = self.fc(out)
        return out


In [23]:
learning_rate = 0.0001
epochs = 20

# 클래스 불균형 가중치 설정 (예: 2번째 클래스에 더 높은 가중치)
weights_tensor = torch.tensor([1/9727, 1/5801], dtype=torch.float).to(device)

# 손실함수 정의 (가중치 적용)
criterion = nn.CrossEntropyLoss(weight=weights_tensor) #변경된 가중치를 불러옴

# 옵티마이저 정의 (예: Adam)
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate,weight_decay=1e-4)

In [24]:
train_batch = len(train_loader)
val_batch = len(val_loader)

In [25]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

print('Learning started.')

# 스케줄러 정의 (성능 기반 자동 감소)-> 
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.8, patience=2, verbose=True)

train_loss_list = []
train_acc_list = []

val_loss_list = []
val_acc_list = []

best_val_loss = 10**9
patience_limit = 5
patience_check = 0

for epoch in range(epochs):
    train_loss = 0
    correct = 0

    model.train()
    for X, Y in train_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        Y_pred = model(X)
        loss = criterion(Y_pred, Y)
        loss.backward()
        optimizer.step()

        train_loss += loss / train_batch
        correct += (torch.argmax(Y_pred, 1) == Y).sum()

    train_acc = (100 * correct / len(train_loader.dataset))
    train_acc_list.append(train_acc.item())
    train_loss_list.append(train_loss.item())

    # validation
    model.eval()
    with torch.no_grad():
        val_loss = 0
        correct = 0

        for X, Y in val_loader:
            X = X.to(device)
            Y = Y.to(device)

            Y_pred = model(X)
            loss = criterion(Y_pred, Y)
            val_loss += loss / val_batch
            correct += (torch.argmax(Y_pred, 1) == Y).sum()

        val_acc = (100 * correct / len(val_loader.dataset))
        val_acc_list.append(val_acc.item())
        val_loss_list.append(val_loss.item())

    # 스케줄러 적용 (val loss 기준)
    scheduler.step(val_loss.item())

    print("[Epoch %2d] loss %.4f  acc %.2f, val loss %.4f  val acc %.2f"
          % (epoch + 1, train_loss, train_acc, val_loss, val_acc))

    # Early stopping
    if val_loss.item() > best_val_loss:
        patience_check += 1
        if patience_check >= patience_limit:
            print('Early Stopping!')
            break
    else:
        best_val_loss = val_loss.item()
        patience_check = 0 

print('Learning Finished!')  

Learning started.
[Epoch  1] loss 0.1400  acc 94.40, val loss 1.4355  val acc 66.77
[Epoch  2] loss 0.0960  acc 96.25, val loss 1.4984  val acc 63.47
[Epoch  3] loss 0.0813  acc 96.95, val loss 1.6239  val acc 64.59
Epoch 00004: reducing learning rate of group 0 to 8.0000e-05.
[Epoch  4] loss 0.0681  acc 97.57, val loss 2.4586  val acc 64.70
[Epoch  5] loss 0.0460  acc 98.30, val loss 2.3389  val acc 65.15
[Epoch  6] loss 0.0441  acc 98.43, val loss 1.7177  val acc 66.35
Early Stopping!
Learning Finished!
